### Data Ingestion Experiment

In [1]:
import os

In [2]:
# Current directory
%pwd

'c:\\Users\\USER\\DeepLearning\\MLOps\\End-to-end-ML-Project-with-MLflow\\research'

In [3]:
# Move one foolder back(change directory)
os.chdir("../") 

In [4]:
%pwd

'c:\\Users\\USER\\DeepLearning\\MLOps\\End-to-end-ML-Project-with-MLflow'

### Entity setup

In [5]:
from dataclasses import dataclass
from pathlib import Path

In [6]:
@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path
    

### src/mlProject/config/configuration.py
Configuration Manager Class

In [7]:
from src.mlProject.constants import *
from src.mlProject.utils.common import read_yaml, create_directories

In [8]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config
        

### Components for Data Ingestion

In [9]:
import os 
import urllib.request as request
import zipfile
from src.mlProject import logger
from src.mlProject.utils.common import get_size

In [10]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")


    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
            

### Data Ingestion Pipeline

In [11]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-06-10 16:36:48,155: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-06-10 16:36:48,171: INFO: common: yaml file: params.yaml loaded successfully]
[2024-06-10 16:36:48,176: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-06-10 16:36:48,178: INFO: common: created directory at: artifacts]
[2024-06-10 16:36:48,180: INFO: common: created directory at: artifacts/data_ingestion]
[2024-06-10 16:36:49,879: INFO: 1315658430: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 26148
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "e5fe38a7e8ce6dcc96c07de7f4012acb6eea4eda26b0b9963e20e230467292b8"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 100A:E9500:5770:6596:6666DAC7
Accept-Ranges: bytes
Date: Mon, 10 Jun 